In [66]:
import json
import os
import pandas as pd

In [80]:
def read_atributes(file):

    atributos = {}
    number_atrib = 0
    remove = ['nFLows']

    with open(file, 'r') as signature_file:
        signature = json.load(signature_file)
        for step in signature['signature']:
            for atribute in signature['signature'][step]:
                if atribute not in remove:
                    if signature['signature'][step][atribute]['operator'] == 'equal':
                        atributos.setdefault(atribute, signature['signature'][step][atribute]['value_max'])
                    if signature['signature'][step][atribute]['operator'] == 'interval':
                        atributos.setdefault(atribute, signature['signature'][step][atribute]['value'])

    return atributos

In [81]:
sig_path = '/home/amanda/Desktop/bruteforce/'

In [69]:
files = [os.path.join(sig_path, f) for f in os.listdir(sig_path) if os.path.isfile(os.path.join(sig_path, f))]

In [109]:
def query(atributos, tabela):
    sql = 'select count(*) from ' + tabela + ' where '
    statement = ''
    cont = 0
    queries = []
        
    for atrib_name, atrib_value in atributos.items():
        statement += str(atrib_name) + '=' + str(atrib_value)
        if cont < len(atributos.items()) - 1:
            statement += ' AND '
        cont = cont + 1
    print('\n'+statement+'\n')
    
    minimo = 1
    for interval in range(1000,51000,1000):
        sufixo = ' AND idx between ' + str(minimo) + ' AND ' + str(interval) + ' group by IPV4_SRC_ADDR;'
        minimo = interval
        queries.append(sql + statement + sufixo)
        print(sql + statement + sufixo)

    return queries

In [110]:
queries = []
for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos = read_atributes(file)
    queries.append(query(atributos, 'bruteforce_flows'))



Assinatura: /home/amanda/Desktop/bruteforce/bf1.sig

srcport=58644 AND destport=22 AND proto=6

select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 1 AND 1000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 1000 AND 2000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 2000 AND 3000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 3000 AND 4000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 4000 AND 5000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6 AND idx between 5000 AND 6000 group by IPV4_SRC_ADDR;
select count(*) from bruteforce_flows where srcport=58644 AND des

In [87]:
import MySQLdb
def consultar(queries):
    db = MySQLdb.connect("localhost","root","1nstalation","ntopng")
    cursor = db.cursor()
    for query in queries:
        try:
            cursor.execute(query)
            results = cursor.fetchall()
        except:
            print ("Error: unable to fecth data")
    db.close()

'select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6;'

In [88]:
queries

['select count(*) from bruteforce_flows where srcport=58644 AND destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND nBytesSrc=164 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where srcport=60638 AND destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where srcport=60438 AND destport=22 AND proto=6;',
 'select count(*) from bruteforce_flows where srcport=59620 AND destport=22 AND nBytesSrc=164 AND proto=6;',
 'select count(*) from bruteforce_flows where srcport=60606 AND destport=22 AND nPktsSrc=3 AND proto=6;',
 'select count(*) from bruteforce_flows where destport=22 AND proto=6;']

In [92]:
minimo = 1
for interval in range(1000,51000,1000):
    print(minimo,interval)
    minimo = interval

1 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
